In [ ]:
!pip install pytorch-pretrained-bert
!pip install tensorflow-gpu==1.15

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def get_from_drive (file_id,file_name):
  # Authenticate and create the PyDrive client.
  # This only needs to be done once per notebook.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Download a file based on its file ID.
  #
  # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(file_name)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas
dataset_DF = pandas.read_csv("/content/drive/MyDrive/sentipolc_train_processed.csv")
print(dataset_DF)
print("colonne:   ")
print(dataset_DF.columns)

      Unnamed: 0  ...                                              clean
0              0  ...  intanto la partita per via nazionale si compli...
1              1  ...  false illusioni sgradevoli realtà mario monti ...
2              2  ...  false illusioni sgradevoli realtà <hashtag> ed...
3              3  ...  mario monti berlusconi risparmi all italia il ...
4              4  ...  mario monti berlusconi risparmi all italia il ...
...          ...  ...                                                ...
7405        7405  ...  che ci frega di mario monti noi abbiamo mario ...
7406        7406  ...  strepitoso il titolo in prima di libero sul go...
7407        7407  ...  <user> consolati il governo <hashtag> monti </...
7408        7408  ...  <user> beh beate loro ! io nn possiedo nulla d...
7409        7409  ...  caro <hashtag> renzi </hashtag> se <hashtag> g...

[7410 rows x 7 columns]
colonne:   
Index(['Unnamed: 0', 'idtwitter', 'opos', 'oneg', 'text', 'label', 'clean'], dtype='obj

In [ ]:
import time
dataset_reduced = dataset_DF
start_time = time.time()
lista_text_tokenized = []
lista_lunghezze = []
i = 0 #per tenere conto dell'indice del plot
over_128 = 0 #conteggio dei plots che superano i 512 tokens

for text in dataset_reduced["clean"]:
  i+=1
  marked_text=text
  tokenized_text = marked_text.split(" ")
  #print(tokenized_text)
  lista_text_tokenized.append(tokenized_text)
  lunghezza = len(tokenized_text)
  lista_lunghezze.append(lunghezza)
  #stampiamo solo i plots che eccedono nella lunghezza massima, pari a 512
  if lunghezza > 128:
    over_128+=1
  if (i%100)==0:
    print(str(i) + " text tokenizzati")

print(lista_text_tokenized)

100 text tokenizzati
200 text tokenizzati
300 text tokenizzati
400 text tokenizzati
500 text tokenizzati
600 text tokenizzati
700 text tokenizzati
800 text tokenizzati
900 text tokenizzati
1000 text tokenizzati
1100 text tokenizzati
1200 text tokenizzati
1300 text tokenizzati
1400 text tokenizzati
1500 text tokenizzati
1600 text tokenizzati
1700 text tokenizzati
1800 text tokenizzati
1900 text tokenizzati
2000 text tokenizzati
2100 text tokenizzati
2200 text tokenizzati
2300 text tokenizzati
2400 text tokenizzati
2500 text tokenizzati
2600 text tokenizzati
2700 text tokenizzati
2800 text tokenizzati
2900 text tokenizzati
3000 text tokenizzati
3100 text tokenizzati
3200 text tokenizzati
3300 text tokenizzati
3400 text tokenizzati
3500 text tokenizzati
3600 text tokenizzati
3700 text tokenizzati
3800 text tokenizzati
3900 text tokenizzati
4000 text tokenizzati
4100 text tokenizzati
4200 text tokenizzati
4300 text tokenizzati
4400 text tokenizzati
4500 text tokenizzati
4600 text tokenizza

In [ ]:
i = 0
for tokenized_plot in lista_text_tokenized:
  if len(tokenized_plot) >= 127:
    tokenized_text_reduced = tokenized_plot[:127]
    tokenized_text_reduced.append("[SEP]")
    lista_text_tokenized[i] = tokenized_text_reduced
  i+=1

In [ ]:
#Load WMAL
import pandas
wmal = pandas.read_csv("SENTIPOLC_WMAL.csv")
wmal

,id,words,value
0,1,governo,0.000000
1,1,minuti,0.000000
2,1,piano,-0.715326
3,1,presentazione,0.000000
4,1,scuola,-0.293017
...,...,...,...
15450,1982,sentirò,-0.006040
15451,1982,spesso,0.000000
15452,1982,stare,0.221995
15453,1982,vedi,0.280564


In [ ]:
wmal_dic ={}
for index, item in wmal.iterrows():
  wmal_dic[item["words"]] = item["value"]

In [ ]:
#Create matrix
import numpy as np
wmal_matrix = []
for tokenized_plot in lista_text_tokenized:
  text =[]
  for tok in tokenized_plot:
    try:
      score = wmal_dic[tok]
      text.append(score)
    except:
      text.append(0)
  text = np.array(text)
  for a in range(text.shape[0],128):
    text=np.insert(text,-1,0)
  wmal_matrix.append(text)

In [ ]:
wmal_matrix=np.array(wmal_matrix)
wmal_matrix.shape
wmal_matrix

array([[ 0.       ,  0.       , -0.5085923, ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
        -0.5418907],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.7232401,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [ ]:
import joblib
joblib.dump(wmal_matrix,"drive/MyDrive/wmal_train_7410_128.joblib")

['drive/MyDrive/wmal_train_7410_128.joblib']